### Instructions::

#### Step 1: `pip install gspread oauth2client`

#### Step 2: Generate service account file from [google developers console](http://gspread.readthedocs.io/en/latest/oauth2.html).  

#### Step 3: Enable sheets api and drive api in Google API console.

#### Step 4: Create a spreadsheet and share it with the `client_email` in the service account credentials file

In [52]:
import gspread
from pprint import pprint
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import re

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)         

gc = gspread.authorize(credentials)

#### Authorization is done. Now you can access all sheets accessible to the `client_email` in the sevice account file.

#### You can open any spreadsheet using the it's name or directly by it's url
   
    gc.open('myspreadsheetname')
    
    or
    
    gc.open_by_url('https://docs.google.com/spreadsheets/d/1d0wo8_-9q7AzUJttgIJfipUHNwpFPC1Wn5QitY7jUhw/edit#gid=0')

In [9]:
sheet = gc.open('gspread_test') # I have made a test spreadsheet for this guide with the name 'gspread_test'

A typical sheet has the following functinons that would be useful for you:
`'add_worksheet',
   'batch_update',
   'client',
   'del_worksheet',
   'fetch_sheet_metadata',
   'get_worksheet',
   'id',
   'list_permissions',
   'remove_permissions',
   'share',
   'sheet1',
   'title',
   'updated',
   'values_append',
   'values_clear',
   'values_get',
   'values_update',
   'worksheet',
   'worksheets'`

You can get more info about any of this functions by running `help(sheet.func_name)`

In [13]:
sheet.worksheets() # This returns the active workshseets. I currently have a single sheet by default.

[<Worksheet 'Sheet1' id:0>]

In [17]:
wks = sheet.sheet1 # Another way to access the worksheet

A worksheet has following functions for you to use:
`'acell',
 'add_cols',
 'add_rows',
 'append_row',
 'cell',
 'clear',
 'client',
 'col_count',
 'col_values',
 'delete_row',
 'export',
 'find',
 'findall',
 'get_all_records',
 'get_all_values',
 'id',
 'insert_row',
 'range',
 'resize',
 'row_count',
 'row_values',
 'spreadsheet',
 'title',
 'update_acell',
 'update_cell',
 'update_cells',
 'update_title',
 'updated'
` 

You can get more info about any of this functions by running `help(wks.func_name)`

In [18]:
help(wks.insert_row)

Help on method insert_row in module gspread.models:

insert_row(values, index=1, value_input_option='RAW') method of gspread.models.Worksheet instance
    Adds a row to the worksheet at the specified index
    and populates it with values.
    
    Widens the worksheet if there are more values than columns.
    
    :param values: List of values for the new row.
    :param value_input_option: Determines how input data should be
                               interpreted. See `ValueInputOption`_
                               in the Sheets API.
    
    .. _ValueInputOption: https://developers.google.com/sheets/api/reference/rest/v4/ValueInputOption



In [42]:
wks.insert_row(['1', 'Akshay Soni', '2015BEC123', 'B.Tech', 'B Div'], index=1) # Entered 1 row

{'spreadsheetId': '1d0wo8_-9q7AzUJttgIJfipUHNwpFPC1Wn5QitY7jUhw',
 'updatedRange': 'Sheet1!A1:E1',
 'updatedRows': 1,
 'updatedColumns': 5,
 'updatedCells': 5}

In [43]:
wks.get_all_values() # This will return all the rows in the spreedsheet

[['1', 'Akshay Soni', '2015BEC123', 'B.Tech', 'B Div']]

In [44]:
# Let's add some more values
wks.insert_row(['2', 'Pranav Gajjewar', '2015BCS025', 'B.Tech', 'A Div'], index=2)
wks.insert_row(['3', 'Nikhil Chaudhari', '2015BCS033', 'B.Tech', 'A Div'], index=3)
wks.insert_row(['4', 'Pratik Somawanshi', '2015BCS123', 'B.Tech', 'A Div'], index=4)
wks.insert_row(['5', 'Ankur Narkhede', '2015BCS037', 'B.Tech', 'A Div'], index=5)

{'spreadsheetId': '1d0wo8_-9q7AzUJttgIJfipUHNwpFPC1Wn5QitY7jUhw',
 'updatedRange': 'Sheet1!A5:E5',
 'updatedRows': 1,
 'updatedColumns': 5,
 'updatedCells': 5}

You can use the `pandas.DataFrame` to properly formalize these rows as table

In [45]:
table = pd.DataFrame(data=wks.get_all_values(), columns=['Id', 'Name', 'Reg.no', 'Class', 'Div'])
table.set_index('Id')

,Name,Reg.no,Class,Div
Id,,,,
1,Akshay Soni,2015BEC123,B.Tech,B Div
2,Pranav Gajjewar,2015BCS025,B.Tech,A Div
3,Nikhil Chaudhari,2015BCS033,B.Tech,A Div
4,Pratik Somawanshi,2015BCS123,B.Tech,A Div
5,Ankur Narkhede,2015BCS037,B.Tech,A Div


#### How to find a specific cell and its corresponding row?

Use the functions `wks.find` and `wks.row_values` as follows.

In [64]:
row_index = wks.find('2015BCS025')
wks.row_values(row_index.row)

['2', 'Pranav Gajjewar', '2015BCS025', 'B.Tech', 'A Div']

In [65]:
row_indices = wks.findall('A Div')
for i in row_indices:
    pprint(wks.row_values(i.row))

['2', 'Pranav Gajjewar', '2015BCS025', 'B.Tech', 'A Div']
['3', 'Nikhil Chaudhari', '2015BCS033', 'B.Tech', 'A Div']
['4', 'Pratik Somawanshi', '2015BCS123', 'B.Tech', 'A Div']
['5', 'Ankur Narkhede', '2015BCS037', 'B.Tech', 'A Div']


In [74]:
row, col = 4, 2
wks.update_cell(row, col, 'Pratik Somwanshi')

{'spreadsheetId': '1d0wo8_-9q7AzUJttgIJfipUHNwpFPC1Wn5QitY7jUhw',
 'updatedRange': 'Sheet1!B4',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [75]:
row_indices = wks.findall('A Div')
for i in row_indices:
    pprint(wks.row_values(i.row))

['2', 'Pranav Gajjewar', '2015BCS025', 'B.Tech', 'A Div']
['3', 'Nikhil Chaudhari', '2015BCS033', 'B.Tech', 'A Div']
['4', 'Pratik Somwanshi', '2015BCS123', 'B.Tech', 'A Div']
['5', 'Ankur Narkhede', '2015BCS037', 'B.Tech', 'A Div']
